# Initializing the SysML v2 API

In [4]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json
from datetime import datetime

#host = "<specify protocol://host:port of the server that is a provider of the SysML v2 REST/HTTP API"
host = "https://localhost:9000"

ModuleNotFoundError: No module named 'pandas'

# Get projects

In [3]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    projects_data = list(map(lambda b: {'Project Name':b['name'], 'Project ID':b['@id']}, projects))
    df = pd.DataFrame.from_records(projects_data).style.hide(axis='index')
    # if len(projects_data) > 0:
    #     df = df.style.sort_values(by='Project Name')
    display(df)
else:
    pprint("Problem in fetching projects")

ConnectionError: HTTPSConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /projects (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f11210a9180>: Failed to establish a new connection: [Errno 111] Connection refused'))

# Create a new project

In [5]:
timestamp = datetime.now()
project_name = f"Spacecraft project with Element CRUD - {timestamp}"
project_data = {
  "@type":"Project",
  "name": project_name,
  "description": "Spacecraft project demonstrating Element Create, Update, Delete steps"
}

project_post_url = f"{host}/projects" 

project_post_response = requests.post(project_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(project_data))

project_id = ""

if project_post_response.status_code == 200:
    project_response_json = project_post_response.json()
    pprint(project_response_json)
    project_id = project_response_json['@id']
    project_name = project_response_json['name']
else:
    pprint(f"Problem in creating a new Spacecraft project at {timestamp}")
    pprint(project_post_response)

{'@id': 'f0594c9c-55ef-49f6-9591-d27d12cc0afb',
 '@type': 'Project',
 'defaultBranch': {'@id': 'd84e4540-56c8-485a-bace-f91b3620ada5'},
 'description': 'Spacecraft project demonstrating Element Create, Update, '
                'Delete steps',
 'name': 'Spacecraft project with Element CRUD - 2022-09-12 13:16:03.708730'}


# Get branches

In [6]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches of Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
main,d84e4540-56c8-485a-bace-f91b3620ada5,None


# Create 1st Commit on the main (default) branch that creates 2 elements

In [7]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Spacecraft System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Payload System"
      }
    }
  ]
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit1_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit1_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '25853b78-df3e-44d3-9a63-c3221743d82d',
 '@type': 'Commit',
 'owningProject': {'@id': 'f0594c9c-55ef-49f6-9591-d27d12cc0afb'},
 'previousCommit': None,
 'timestamp': '2022-09-12T13:16:13.843497-04:00'}


In [8]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit1_id}/elements" 

element_get_response = requests.get(element_get_url)
payload_system_element_id = ""
spacecraft_system_element_id = ""

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    
    # Gather the id of the Payload System that is created
    payload_system_element = list(filter(lambda e: e['name'] == "Payload System", elements))
    if payload_system_element:
        payload_system_element_id = payload_system_element[0]['@id']
    
    # Gather the id of the Spacecraft System that is created
    spacecraft_system_element = list(filter(lambda e: e['name'] == "Spacecraft System", elements))
    if spacecraft_system_element:
        spacecraft_system_element_id = spacecraft_system_element[0]['@id']
        
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit1_id}.")
    pprint(element_get_response)

Element Name,Element ID
Payload System,1093a5fd-5a5e-47d7-8bfd-b870ede05648
Spacecraft System,07a9a21c-2070-4d6b-8818-bb3bca62dfd8


# Create the 2nd Commit that updates the name of one of the elements

In [9]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"New Payload System",
        "identifier":payload_system_element_id
      },
      "identity": {
        "@id": payload_system_element_id
      }
    }
  ],
  "previousCommit": {
      "@id": commit1_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit2_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit2_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': 'da8991ce-1124-493b-8f34-a4bb2d0800e1',
 '@type': 'Commit',
 'owningProject': {'@id': 'f0594c9c-55ef-49f6-9591-d27d12cc0afb'},
 'previousCommit': {'@id': '25853b78-df3e-44d3-9a63-c3221743d82d'},
 'timestamp': '2022-09-12T13:16:21.941689-04:00'}


# Get all elements after the 2nd commit (should show 1 element updated)

In [10]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}.")
    pprint(element_get_response)

Element Name,Element ID
New Payload System,1093a5fd-5a5e-47d7-8bfd-b870ede05648
Spacecraft System,07a9a21c-2070-4d6b-8818-bb3bca62dfd8


# Create the 3rd commit that deletes an element (same one whose name was updated)

In [11]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload":None,
      "identity": {
        "@id": payload_system_element_id
      }
    }
  ],
  "previousCommit": {
      "@id": commit2_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit3_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit3_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '00f0b229-7e70-4688-80e8-ae2eafc0ce8e',
 '@type': 'Commit',
 'owningProject': {'@id': 'f0594c9c-55ef-49f6-9591-d27d12cc0afb'},
 'previousCommit': {'@id': 'da8991ce-1124-493b-8f34-a4bb2d0800e1'},
 'timestamp': '2022-09-12T13:16:45.992847-04:00'}


# Get all elements after the 3rd commit (should only 1 element)

In [12]:
pprint(f"Fetching elements at commit {commit2_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit3_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}")
    pprint(element_get_response)

'Fetching elements at commit da8991ce-1124-493b-8f34-a4bb2d0800e1'


Element Name,Element ID
Spacecraft System,07a9a21c-2070-4d6b-8818-bb3bca62dfd8
